In [ ]:
import numpy as np

txt = np.loadtxt('C:/Users/edu/venv/Scripts/concrete.csv', delimiter = ',',skiprows = 1, dtype = np.float32)

txt[:,:8]

txt[:,8:9]


In [ ]:
import pandas as pd
df = pd.DataFrame(txt)
df.head()

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() #버젼 2를 1처럼 사용하기 위함

In [ ]:
x_data = txt[:,0:-1] #txt[:,:8]
y_data = txt[:,[-1]] #txt[:,8:9]
x = tf.placeholder(tf.float32, shape = (None,8)) # ->(None, 3)
w = tf.Variable(tf.random_normal([8,1]), name = 'weight')
b = tf.Variable(tf.random_normal([1]), name = 'bias')
Y = tf.placeholder(tf.float32,shape = [None,1]) # -> shape = (None,1)

In [ ]:
hypothesis = tf.matmul(x,w) + b 
cost = tf.reduce_mean(tf.square(hypothesis-Y))

In [ ]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 1e-10)
train = optimizer.minimize(cost)
# 위 코드를 실행시키기 위해 세션을 만듦
with tf.Session() as sess:
    # 세션에 올라가는 변수 초기화
    sess.run(tf.global_variables_initializer())
    for step in range(200001):
        cost_val, w_val, b_val, _ = \
        sess.run([cost, w, b, train], feed_dict={x:x_data, Y:y_data})
        if step % 100000 == 0:
            print(step,"Cost: " ,cost_val,"\nPrediction: ", w_val, "\n",b_val)
    predict = sess.run(hypothesis, feed_dict = {x:x_data})
#reshape(-1) -> (2,3,3)에 적용을 한다면 (2,9)가 된다. <차원의 축소>

    #밑의 코드는 1차원으로 변형 / 상관계수를 구함
    print("Person Correlation Coefficient : ",np.corrcoef(predict.reshape(-1),y_data.reshape(-1)))

    print('strength wiil be ',sess.run(hypothesis, feed_dict = {x : [[141.3,212,0,203.5,0,971.8,748.5,28],[168.9,42.2,124.3,158.3,10.8,1080.8,796.2,14]]}))

- 현재는 각 크고 작은 값들에 가중치가 가하는 영향력이 매우 다르기 때문에 스케일이 필요하다.
- 첫번쨰 행을 예시로 w1 x 141, w2 x 212 ......
- 현재는 평균이 0이고 표준편차가 1인 곳에서 랜덤하게 값을 뽑아오고 있다.(w를)
- 가중치의 스케일이 비슷해야 하는데 이것이 다르므로 y에 미치는 영향이 각각 매우 다르다
- min-max scale 또는 z-score 사용하여 스케일링을 try
- 데이터가 많이 편향되어 있으면 제대로 된 예측을 할 수 없다

- 랜덤하게 데이터를 뽑을 때 Truncated Normal(좁고 뚱뚱한 분포모양) / Uniform(직사가형 모양)도 있다.


### 로지스틱 회귀
- 종속변수의 값을 0 또는 1로 표현할 수 있는 경우 사용
- 수치형 데이터는 거리를 계산하는데 필요
- 훈련하는데 대부분의 시간이 소모되며, 이 단계에서 데이터를 분류하기 위해 최적의 계수를 찾음
- 훈련단계를 마치고 나면 분류는 빠르고 쉽게 진행됨

- 장점:
    - 계산 비용이 적다
    - 구현이 쉽다
    - 결과 해석을 위한 지식 표현이 쉽다
- 단점:
    - Underfitting이 주로 됌


- 0과 1의 결과를 갖는 그래프는 직선회귀보다는 S-shape 함수에 더 가깝다
- 이런 S-shape의 모양을 가지는 함수가 바로 Sigmoid 함수 (로지스틱 회귀 분류기) 이다.
---------------------------------
- 로지스틱 회귀 분류기
- 각각의 속성에 가중치를 곱한 다음 서로 더한 뒤, 시그모이드 함수에 넣고 0에서 1 사이의 수를 구한다.
- 이 수가 0.5 이상이면 1로, 이하면 0으로 분류한다.


In [ ]:
tf.set_random_seed(777)

x_data = [[1,2],
          [2,3],
          [3,1],
          [4,3],
          [5,3],
          [6,2]]
y_data = [[0],
          [0],
          [0],
          [1],
          [1],
          [1]]

x = tf.placeholder(tf.float32, shape = (None,2))
Y = tf.placeholder(tf.float32,shape = [None,1]) 

w = tf.Variable(tf.random_normal([2,1]), name = 'weight')
b = tf.Variable(tf.random_normal([1]), name = 'bias')


#hypothesis는 시그모이드 적용하므로 0~1이다
hypothesis = tf.sigmoid(tf.matmul(x,w)+b)  #시그모이드를 적용

#cost를 현재 살펴보면 (0~1)(0~1)의 숫자들을 제곱하는 것이므로 더 작은 값이 된다.
#cost가 점점 커져야 하는데 작아지므로 의도와 맞지 않는다. (mean squared error)
#현재의 현황을 그래프로 그리면 단조감소인데 이것을 해결하기 위해 LOG를 취한다.

#cost = tf.reduce_mean(tf.square(hypothesis- Y))

#따라서 크로스 엔트로피를 적용한다.
#간단한 예시로 Y가 1인데 hypothesis가 0으로 예측을 했다면 밑의 수식에 대입을 하면 
#log0이 나오게 되고 이는 무한대이다. 즉 틀리면 무한대, 맞으면 0으로 산출된다.
cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1-hypothesis))

optimizer = tf.train.GradientDescentOptimizer(learning_rate= 1e-3)

train = optimizer.minimize(cost)

#predicted에 0과 1만 들어가게 된다.
predicted = tf.cast(hypothesis>0.5, dtype=tf.float32)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step in range(2000001):
    cost_val, pred, _ = \
    sess.run([cost, predicted, train], feed_dict={x:x_data, Y:y_data})
    if step % 1000000 == 0:
        print(step,"Cost: " ,cost_val,"\nPrediction: ", pred)
        
        #맞으면 1 아니면 0을 평균 낸다.
        print("Accuracy : ", np.mean(np.where(pred == y_data,1,0)))

In [ ]:
data = np.loadtxt('C:/Users/edu/venv/Scripts/data-03-diabetes.csv', delimiter = ',',skiprows = 1, dtype = np.float32)

In [ ]:
x_data = data[:,0:-1] #txt[:,:8]
y_data = data[:,[-1]] #txt[:,8:9]
x = tf.placeholder(tf.float32, shape = (None,8)) # ->(None, 3)
w = tf.Variable(tf.random_normal([8,1]), name = 'weight')
b = tf.Variable(tf.random_normal([1]), name = 'bias')
Y = tf.placeholder(tf.float32,shape = [None,1]) # -> shape = (None,1)

hypothesis = tf.sigmoid(tf.matmul(x,w) + b) 

cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1-hypothesis))

#optimizer = tf.train.GradientDescentOptimizer(learning_rate= 1e-4)

train = tf.train.GradientDescentOptimizer(learning_rate = 0.01).minimize(cost)

#predicted에 0과 1만 들어가게 된다.
predicted = tf.cast(hypothesis>0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted,Y),dtype = tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(10001):
        cost_val, _ = \
        sess.run([cost train], feed_dict={x:x_data, Y:y_data})
        if step % 1000 == 0:
            print(step,"Cost: " ,cost_val)
        
        h, c, a = sess.run([hypothesis, predicted, accuracy],
                          feed_dict = {x:x_data, Y:y_data})
        print('\nHypothesis : ',h, '\nCorrect : ',c,'\nAccuracy : ',a )

### 신경망 이해

- 인공 신경망은 어떻게 생물적으로 뇌가 감각 입력의 자극에 반응하는지에 대해 이해로부터 얻어진 모델
- 입력 신호와 출력 신호 간의 관계를 모델링
- 뇌가 막대한 병렬 프로세서를 생성하기 위해 뉴런이라는 세포로 연결된 망을 사용하듯이 ANN은 학습 문제를 풀기 위해 인공 뉴런이나 노드의 망을 사용
    - 사람 : 850억 뉴런
    - 고양이 : 10억 뉴런
    - 쥐 : 7천 5백만 뉴런
    - 바퀴벌레 : 몇 백만 뉴런

In [ ]:
#ANN 신경망
#input layer는 데이터의 속성의 갯수
#output layer는 모델을 만든 목적에 따라 결정된다.
#즉 클래스를 예측을 할 것이면 output은 많아진다.
#수치 예측 모델이면 output은 1개이다 

tf.set_random_seed(777)

x_data = [[0,0],
          [0,1],
          [1,0],
          [1,1]]
y_data = [[0],
          [1],
          [1],
          [0]]

x = tf.placeholder(tf.float32, shape = (None,2))
Y = tf.placeholder(tf.float32,shape = [None,1]) 

#2개 입력 받아서 2곳으로 출력해야 하므로 2,2
#x1 * w11 + x2 * w21 +b , x1 * w12 + x2 * w22 +b의 식의 모양에서 
#x는 2x1 모양, w는 2x2 모양인 것을 확인할 수 있다.
#각 단계를 입력과 출력으로 생각하여 (입력, 출력)으로 생각하면 쉽다. (노드 그림 그려보면 이해 가능)
w1 = tf.Variable(tf.random_normal([2,2]), name = 'weight1')

#따라서 bias도 두개
b1 = tf.Variable(tf.random_normal([2]), name = 'bias1')
l1 = tf.sigmoid(tf.matmul(x,w1) + b1)

w2 = tf.Variable(tf.random_normal([2,1]), name = 'weight2')
b2 = tf.Variable(tf.random_normal([1]), name = 'bias2')

#l1과 w2를 sigmoid하는 것 생각, bias는 당연히 b2
hypothesis = tf.sigmoid(tf.matmul(l1,w2)+b2)

cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1-hypothesis))

optimizer = tf.train.GradientDescentOptimizer(learning_rate= 1e-3)

train = optimizer.minimize(cost)


predicted = tf.cast(hypothesis>0.5, dtype=tf.float32)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step in range(2001):
    cost_val, pred, _ = \
    sess.run([cost, predicted, train], feed_dict={x:x_data, Y:y_data})
    if step % 1000 == 0:
        print(step,"Cost: " ,cost_val,"\nPrediction: ", pred)
        

        print("Accuracy : ", np.mean(np.where(pred == y_data,1,0)))